In [40]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from itertools import product

In [153]:
# carregar os datasets
cervejas = pd.read_csv('./datasets/raw/CERVEJAS.csv')
harmonizacoes = pd.read_csv('./datasets/preprocessed/HARMONIZACOES_processed.csv')

In [175]:
cervejas[['name','ibumin', 'ibumax', 'ogmin', 'ogmax', 'fgmin', 'fgmax', 'abvmin', 'abvmax', 'srmmin', 'srmmax']]

,name,ibumin,ibumax,ogmin,ogmax,fgmin,fgmax,abvmin,abvmax,srmmin,srmmax
0,Altbier,25,50,1.044,1.052,1.008,1.014,4.3,5.5,9.0,17.0
1,American Amber Ale,25,40,1.045,1.060,1.010,1.015,4.5,6.2,10.0,17.0
2,American Barleywine,50,100,1.080,1.120,1.016,1.030,8.0,12.0,9.0,18.0
3,American Brown Ale,20,30,1.045,1.060,1.010,1.016,4.3,6.2,18.0,35.0
4,American IPA,40,70,1.056,1.070,1.008,1.014,5.5,7.5,6.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...
91,Wee Heavy,17,35,1.070,1.130,1.018,1.040,6.5,10.0,14.0,25.0
92,Weissbier,8,15,1.044,1.053,1.008,1.014,4.3,5.6,2.0,6.0
93,Weizenbock,15,30,1.064,1.090,1.015,1.022,6.5,9.0,6.0,25.0
94,Wheatwine,30,60,1.080,1.120,1.016,1.030,8.0,12.0,6.0,14.0


In [177]:
# criacao de faixas entre os valores de min e max de cada um dos atributos para 
# eliminar as colunas de min e max, ficando apenas 1 coluna por atributo

def criar_faixas(df, num_faixas=3):
    faixas = []
    for _, row in df.iterrows():
        for i in range(num_faixas):
            faixa = {
                'estilo': row['name'],
                'categoria': row['category'],
                'ibu': np.linspace(row['ibumin'], row['ibumax'], num_faixas)[i],
                'og': np.linspace(row['ogmin'], row['ogmax'], num_faixas)[i],
                'fg': np.linspace(row['fgmin'], row['fgmax'], num_faixas)[i],
                'abv': np.linspace(row['abvmin'], row['abvmax'], num_faixas)[i],
                'srm': np.linspace(row['srmmin'], row['srmmax'], num_faixas)[i]
            }
            faixas.append(faixa)
    return pd.DataFrame(faixas)

In [179]:
# criando faixas de parâmetros com valores originais
cervejas = criar_faixas(cervejas)

In [189]:
cervejas[['estilo','ibu', 'og', 'fg', 'abv', 'srm']]

,estilo,ibu,og,fg,abv,srm
0,Altbier,25.0,1.0440,1.0080,4.30,9.0
1,Altbier,37.5,1.0480,1.0110,4.90,13.0
2,Altbier,50.0,1.0520,1.0140,5.50,17.0
3,American Amber Ale,25.0,1.0450,1.0100,4.50,10.0
4,American Amber Ale,32.5,1.0525,1.0125,5.35,13.5
...,...,...,...,...,...,...
283,Wheatwine,45.0,1.1000,1.0230,10.00,10.0
284,Wheatwine,60.0,1.1200,1.0300,12.00,14.0
285,Witbier,8.0,1.0440,1.0080,4.50,2.0
286,Witbier,14.0,1.0480,1.0100,5.00,3.0


In [165]:
# Definir as colunas dos atributos das cervejas que queremos normalizar
range_atributos_cerveja = ['ibumin', 'ibumax', 'ogmin', 'ogmax', 'fgmin', 'fgmax', 'abvmin', 'abvmax', 'srmmin', 'srmmax']
atributos_cerveja = ['ibu', 'og', 'fg', 'abv', 'srm']

In [191]:
# gerando todas as possíveis combinacoes entre os valores de faixa para os atributos
# cada linha gerada simula um possível rótulo de cerveja para um estilo

def gerar_combinacoes(df_faixas):
    faixas_combinadas = []
    for estilo in df_faixas['estilo'].unique():
        sub_df = df_faixas[df_faixas['estilo'] == estilo]
        combinacoes = product(sub_df['ibu'], sub_df['og'], sub_df['fg'], sub_df['abv'], sub_df['srm'])
        
        for combo in combinacoes:
            faixa_comb = {
                'estilo': estilo,
                'ibu': combo[0],
                'og': combo[1],
                'fg': combo[2],
                'abv': combo[3],
                'srm': combo[4]
            }
            for col in sub_df.columns:
                if col not in faixa_comb:
                    faixa_comb[col] = sub_df[col].values[0]  # Acessa o valor correspondente ao estilo
            faixas_combinadas.append(faixa_comb)
    
    return pd.DataFrame(faixas_combinadas)

In [193]:
# gerando as combinações a partir das faixas
cervejas = gerar_combinacoes(cervejas)

In [201]:
cervejas_altbier = cervejas[cervejas['estilo'] == 'Altbier'][['estilo', 'ibu', 'og', 'fg', 'abv', 'srm']]
cervejas_altbier

,estilo,ibu,og,fg,abv,srm
0,Altbier,25.0,1.044,1.008,4.3,9.0
1,Altbier,25.0,1.044,1.008,4.3,13.0
2,Altbier,25.0,1.044,1.008,4.3,17.0
3,Altbier,25.0,1.044,1.008,4.9,9.0
4,Altbier,25.0,1.044,1.008,4.9,13.0
...,...,...,...,...,...,...
238,Altbier,50.0,1.052,1.014,4.9,13.0
239,Altbier,50.0,1.052,1.014,4.9,17.0
240,Altbier,50.0,1.052,1.014,5.5,9.0
241,Altbier,50.0,1.052,1.014,5.5,13.0


In [149]:
# verificando balanceamento por estilo e por categoria

# Contagem de linhas por categoria (verificação) e ordenação
row_count_per_estilo = cervejas['estilo'].value_counts().sort_values(ascending=False)
row_count_per_category = cervejas['categoria'].value_counts().sort_values(ascending=False)

# Contagem de estilos únicos por categoria (como antes) e ordenação
# unique_styles_per_category = df_novo_harmonizacao.groupby('category')['estilos_cervejas'].nunique().sort_values(ascending=False)

# Exibir os resultados
print("Número total de linhas por categoria (ordenado do maior para o menor):\n", row_count_per_estilo)
print("\nNúmero de estilos de cervejas únicos por categoria (ordenado do maior para o menor):\n", row_count_per_category)


Número total de linhas por categoria (ordenado do maior para o menor):
 estilo
Altbier                243
American Amber Ale     243
American Barleywine    243
American Brown Ale     243
American IPA           243
                      ... 
Weissbier              243
Weizenbock             243
Wheatwine              243
Specialty IPA          243
Witbier                243
Name: count, Length: 96, dtype: int64

Número de estilos de cervejas únicos por categoria (ordenado do maior para o menor):
 categoria
Historical Beer                  1944
European Sour Ale                1701
Strong American Ale               972
Standard American Beer            972
Pale Bitter European Beer         972
Dark British Beer                 972
Monastic Ale                      972
Czech Lager                       972
Strong British Ale                972
Strong Belgian Ale                729
International Lager               729
Ipa                               729
Scottish Ale                     

In [205]:
# normalizando os valores para todos se encaixarem entre 0 e 1, respeitando a proporção

# inicializando o normalizador MinMaxScaler
scaler = MinMaxScaler()

cervejas[atributos_cerveja] = scaler.fit_transform(cervejas[atributos_cerveja])

In [211]:
cervejas_altbier = cervejas[cervejas['estilo'] == 'Altbier'][['estilo', 'ibu', 'og', 'fg', 'abv', 'srm']]
cervejas_altbier

,estilo,ibu,og,fg,abv,srm
0,Altbier,0.25,0.173077,0.36,0.163793,0.194805
1,Altbier,0.25,0.173077,0.36,0.163793,0.298701
2,Altbier,0.25,0.173077,0.36,0.163793,0.402597
3,Altbier,0.25,0.173077,0.36,0.215517,0.194805
4,Altbier,0.25,0.173077,0.36,0.215517,0.298701
...,...,...,...,...,...,...
238,Altbier,0.50,0.250000,0.48,0.215517,0.298701
239,Altbier,0.50,0.250000,0.48,0.215517,0.402597
240,Altbier,0.50,0.250000,0.48,0.267241,0.194805
241,Altbier,0.50,0.250000,0.48,0.267241,0.298701


In [124]:
# exportando base tratada de cerveja

cervejas.to_csv('./datasets/preprocessed/CERVEJAS_processed.csv', index=False)